<a href="https://colab.research.google.com/github/Alcaline/models/blob/master/research/object-detection/notebooks/ITD_MobNetv3_TF1_SSD_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

#Check if it's a local runtime
try:
  import google.colab
  IN_COLAB = True
  # if(not tf.VERSION == "1.12.3"):
  #   !pip uninstall -y tensorflow tensorflow-gpu
  #   !pip install "tensorflow-gpu<1.13.0"
  #   !wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
  #   !dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
  #   !apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
  #   !apt-get update
  #   !apt-get install cuda=9.0.176-1
except:
  IN_COLAB = False

Python 3.6.9


Set constants paths

In [ ]:
#Constants 
import os

TEST_RATIO = 0.1
RANDOM_SEED = 337533672

if(IN_COLAB):
  models_path = "/content/models"
  annotations_path = "{}/annotations/xmls".format(models_path)
  labels_path = "{}/annotations".format(models_path)
  images_path = "{}/images".format(models_path)
  data_path = "{}/data".format(models_path)
  checkpoint_path = "{}/model".format(models_path)
else:
  models_path = os.environ['MODELS_PATH']
  annotations_path = os.environ['XMLS_PATH']
  labels_path = os.environ['ANNOTATIONS_PATH']
  images_path = os.environ['IMAGES_PATH']
  data_path = os.environ['DATA_PATH']

Install TF Model Garden and compile protos

In [ ]:
if(IN_COLAB):
  !git clone https://github.com/tensorflow/models.git {models_path}
  !cd {models_path}/research && python setup.py build && python setup.py install

  !cd {models_path}/research/ && protoc object_detection/protos/*.proto --python_out=.
  !pip install tf_slim
  
  %tensorflow_version 1.x

  import os
  pythonpath = os.environ["PYTHONPATH"]
  %set_env PYTHONPATH={pythonpath}:{models_path}/research:{models_path}/research/slim
# else:
#   !cd models && git checkout bdf6f121bdb0c3b0e519d09f3fecb48f9caf98d3


Check Installation

In [ ]:
try:
  import tensorflow.compat.v1 as tf
  from tensorflow.compat.v1 import InteractiveSession
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  session = InteractiveSession(config=config)
  print(tf.version.VERSION)
  !nvcc --version
except:
  from tensorflow import InteractiveSession
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  session = InteractiveSession(config=config)
  print(tf.VERSION)

device_name = tf.test.gpu_device_name()
print(device_name)
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


1.15.2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
/device:GPU:0
Found GPU at: /device:GPU:0


In [ ]:
#@title Mount Drive
mount_dataset = False #@param {"type":"boolean"}

if(IN_COLAB):
  from google.colab import drive
  drive.mount("/mnt")  

  model_tgt_path = "My Drive/models" #@param {type:"string"}
  data_tgt_path = "My Drive/models/data" #@param {type:"string"}
  annotations_tgt_path = "My Drive/models/annotations" #@param {type:"string"}
  dataset_images_tgt_path = "My Drive/Datasets/Industrial Tools Database/ITD Dataset/ColorImages" #@param {type:"string"}
  dataset_xmls_tgt_path = "My Drive/Datasets/Industrial Tools Database/ITD Dataset/Annotations" #@param {type:"string"}
    
  !mkdir -p {data_path}
  !mkdir -p {images_path}
  !mkdir -p {annotations_path}
  !mount --bind '/mnt/{data_tgt_path}' {data_path}
  !mount --bind '/mnt/{annotations_tgt_path}' {labels_path}
  
  if(mount_dataset):
    !mount --bind '/mnt/{dataset_images_tgt_path}' {images_path}
    !mount --bind '/mnt/{dataset_xmls_tgt_path}' {annotations_path}


Load model from name Function

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  file_name = "{}.tar.gz".format(model_name)
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + file_name,
    untar=True)
  return model_dir

In [ ]:
#@title Setup Model
if(IN_COLAB):
  import os
  selected_model = "ssd_mobilenet_v3_small_coco_2020_01_14" #@param ["ssd_mobilenet_v2_coco_2018_03_29", "ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03", "ssd_mobilenet_v3_large_coco_2020_01_14", "ssd_mobilenet_v3_small_coco_2020_01_14", "ssdlite_mobilenet_v2_coco_2018_05_09", "ssd_inception_v2_coco_2018_01_28", "empty"]
  if(not os.path.isdir("/mnt/{}/{}".format(model_tgt_path, selected_model))):
    model_dir = load_model(selected_model)
    !ls {model_dir}
    !cp -r {model_dir} /mnt/My\ Drive/models/
  !mkdir -p {checkpoint_path}
  !umount {checkpoint_path}
  !mount --bind /mnt/My\ Drive/models/{selected_model} {checkpoint_path}
  !mkdir {checkpoint_path}/train
  !mkdir {checkpoint_path}/eval

In [ ]:
#@title Verify Dataset
dataset_size = 11766 #@param{"type":"integer"}

file_name_list = list(range(1, dataset_size))
all_annotations = list(map(lambda x: "{:06d}.xml".format(x), file_name_list))
all_images = list(map(lambda x: "{:06d}.jpg".format(x), file_name_list))

if(mount_dataset):
  annotations_files = os.listdir(annotations_path)
  images_files = os.listdir(images_path)

  print("Annotations size: ", len (annotations_files) )
  print("Images size: ", len (images_files) )

  annotations_paths = list(map(lambda x: "{}/{}".format(annotations_path, x), annotations_files))
  images_paths = list(map(lambda x: "{}/{}".format(images_path, x), images_files))

  annotations_files.sort()
  images_files.sort()
  annotations_paths.sort()
  images_paths.sort()

  missing_annotations = list(set(all_annotations) - set(annotations_files))
  missing_annotations.sort()

  print("Annotations missing: ", len (missing_annotations) )
  print(missing_annotations)

  missing_images = list(set(all_images) - set(images_files))
  missing_images.sort()

  print("Images missing: ", len (missing_images) )
  print(missing_images)
else:
  print("Skipping dataset verification")

Skipping dataset verification


XML to CSV row function

In [ ]:
def parse_from_xml(annotation, image):
  import xml.etree.ElementTree as et
  xtree = et.parse(annotation)
  xroot = xtree.getroot()
  xsize = xroot.find("size")
  img_width = int(xsize.find("width").text)
  img_height = int(xsize.find("height").text)
  img_depth = int(xsize.find("depth").text)
  img_size = (img_width, img_height) #, img_depth)

  def _getDetection(xobj):
    label = xobj.find("name").text

    xbndbox = xobj.find("bndbox")
    xmin = int(xbndbox.find("xmin").text)
    xmax = int(xbndbox.find("xmax").text)
    ymin = int(xbndbox.find("ymin").text)
    ymax = int(xbndbox.find("ymax").text)
    bounding_box = (xmin, xmax, ymin, ymax)

    return (label, bounding_box)
  
  detections = map (_getDetection, xroot.findall('object'))

  return image, img_size, list(detections)

Export to CSV function

In [ ]:
import pandas as pd

def save_csv(fname, parent, data, save_partition, headers=None):
  from csv import writer as w

  dataset_file = fname
  dataset_path = '{}/{}'.format(parent, dataset_file)

  row = []
  for file, size, detection in data:
    (width, height) = size
    for label, bnd in detection:
      (xmin, xmax, ymin, ymax) = bnd
      row.append((file, width, height, label, xmin, xmax, ymin, ymax))

  if(headers != None):
    df = pd.DataFrame(row, columns=headers)
  else:
    df = pd.DataFrame(row)


  if(save_partition):
    df.to_csv(dataset_path, index=False)

  return df

Create partitioned (sharded) dataframe based dataset function

In [ ]:
def create_dataset_partitions(annotations, images, partition_size=0, max_items=0, parent=data_path, overwrite=False, save_partition=False):
  import os

  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'xmax', 'ymin', 'ymax']

  def _do_create(begin, end):
    partition_name = "labels-{:06d}-{:06d}.csv".format(begin+1, end)
    partition_path = '{}/{}'.format(parent, partition_name)

    if (overwrite or not os.path.exists(partition_path)):
      print ("Creating dataset "+ partition_name)
      data = [parse_from_xml(annotation, image) for annotation, image in zip(annotations[begin: end], images[begin: end])]
      df = save_csv(partition_name, parent, data, save_partition, column_name)
      print ("Created dataset "+partition_name)
    else:
      print (partition_name+" exists")
      df = pd.read_csv(partition_path)

    return df


  dataframes = []
  size = len(annotations) if max_items == 0 else max_items
  psize = partition_size if partition_size>0 else size
  partition_count = int(size/psize)
  i = 0
  for i in range(0, partition_count):
    begin = i*psize
    i += 1
    end = i*psize 
    dataframes.append(_do_create(begin, end))

  begin = i*psize
  end = partition_count*psize + size % psize
  if (begin != end):
    dataframes.append(_do_create(begin, end))
  
  dataframe = pd.concat(dataframes)

  return dataframe

In [ ]:
def show_bb(filename, detections, images_dir=images_path):
  from object_detection.utils import visualization_utils as vis_util
  import numpy as np
  from PIL import Image
  import io

  with tf.io.gfile.GFile(os.path.join(images_dir, '{}'.format(filename)), 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image_np = np.array(Image.open(encoded_jpg_io))

  for xmin,xmax,ymin,ymax in detections:
    vis_util.draw_bounding_box_on_image_array(image_np, xmin,ymin,xmax,ymax)

  return image_np

In [ ]:
def get_detections(group):
  return [(row['ymin']/row['height'],
           row['ymax']/row['height'],
           row['xmin']/row['width'], 
           row['xmax']/row['width']) for index, row in group.object.iterrows()]

In [ ]:
#@title Create CSV
overwrite = False #@param{type:'boolean'}

dataframe_file = "labels.csv"
dataframe_path = "{}/{}".format(data_path, dataframe_file)

dataframe = create_dataset_partitions(annotations_paths, images_files, overwrite=overwrite) if overwrite or not os.path.exists(dataframe_path) else pd.read_csv(dataframe_path)

dataframe.to_csv(dataframe_path, index=False)
dataframe

,filename,width,height,class,xmin,xmax,ymin,ymax
0,000001.jpg,960,540,cutting tools,423,547,170,264
1,000002.jpg,960,540,cutting tools,446,554,178,266
2,000003.jpg,960,540,cutting tools,445,555,189,264
3,000004.jpg,960,540,cutting tools,449,561,201,276
4,000005.jpg,960,540,cutting tools,447,559,210,287
...,...,...,...,...,...,...,...,...
11875,011765.jpg,960,540,cutting tools,248,296,363,427
11876,011765.jpg,960,540,cutting tools,210,247,201,277
11877,011765.jpg,960,540,measuring tools,176,212,197,307
11878,011765.jpg,960,540,fastener tools,172,264,317,350


Split dataset in Training and Evaluation

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

validation_only_start_index = 11761 #@param{"type":"integer"}
validation_only = set(['{:06d}.jpg'.format(i) for i in range(validation_only_start_index, dataset_size)])

to_split_dict = {}
validation_only_dict = {}
for index, row in list(dataframe.iterrows()):
  filename = row['filename']
  select_dict = validation_only_dict if filename in validation_only else to_split_dict
  clazz = row['class']
  if filename not in select_dict:
    select_dict[filename] = (set(), set())
  indexes, classes = select_dict[filename]
  indexes.add(index)
  classes.add(clazz)
to_split_dict


{'000001.jpg': ({0}, {'cutting tools'}),
 '000002.jpg': ({1}, {'cutting tools'}),
 '000003.jpg': ({2}, {'cutting tools'}),
 '000004.jpg': ({3}, {'cutting tools'}),
 '000005.jpg': ({4}, {'cutting tools'}),
 '000006.jpg': ({5}, {'cutting tools'}),
 '000007.jpg': ({6}, {'cutting tools'}),
 '000008.jpg': ({7}, {'cutting tools'}),
 '000009.jpg': ({8}, {'cutting tools'}),
 '000010.jpg': ({9}, {'cutting tools'}),
 '000011.jpg': ({10}, {'cutting tools'}),
 '000012.jpg': ({11}, {'cutting tools'}),
 '000013.jpg': ({12}, {'cutting tools'}),
 '000014.jpg': ({13}, {'cutting tools'}),
 '000015.jpg': ({14}, {'cutting tools'}),
 '000016.jpg': ({15}, {'cutting tools'}),
 '000017.jpg': ({16}, {'cutting tools'}),
 '000018.jpg': ({17}, {'cutting tools'}),
 '000019.jpg': ({18}, {'cutting tools'}),
 '000020.jpg': ({19}, {'cutting tools'}),
 '000021.jpg': ({20}, {'cutting tools'}),
 '000022.jpg': ({21}, {'cutting tools'}),
 '000023.jpg': ({22}, {'cutting tools'}),
 '000024.jpg': ({23}, {'cutting tools'}),
 '

In [ ]:
import numpy as np
validation_only_indexes = np.array([list(indexes) for indexes, classes in validation_only_dict.values()]).reshape(-1)
print(validation_only_indexes)

X, y = np.array(list(to_split_dict.values())).transpose().tolist()
X = np.array([list(index_set) for index_set in X])
y = np.array(['mixed' if len(class_set) > 1 else '{}'.format(set(class_set)) for class_set in y])
print(X)
print(y)

sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_RATIO, random_state=RANDOM_SEED)


for train_index, val_index in sss.split(X, y):
  print("TRAIN:", train_index, "TEST:", val_index)
  X_train, X_val = X[train_index].reshape(-1), np.concatenate((X[val_index].reshape(-1), validation_only_indexes))

  train_dataframe, val_dataframe = dataframe.iloc[X_train], dataframe.iloc[X_val]

train_dataframe, val_dataframe

[11776 11777 11778 11779 11780 11781 11782 11783 11760 11761 11762 11763
 11764 11765 11766 11767 11768 11769 11770 11771 11772 11773 11774 11775
 11784 11785 11786 11787 11788 11789 11790 11791 11792 11793 11794 11795
 11796 11797 11798 11799 11800 11801 11802 11803 11804 11805 11806 11807
 11808 11809 11810 11811 11812 11813 11814 11815 11816 11817 11818 11819
 11820 11821 11822 11823 11824 11825 11826 11827 11828 11829 11830 11831
 11832 11833 11834 11835 11836 11837 11838 11839 11840 11841 11842 11843
 11844 11845 11846 11847 11848 11849 11850 11851 11852 11853 11854 11855
 11856 11857 11858 11859 11860 11861 11862 11863 11864 11865 11866 11867
 11868 11869 11870 11871 11872 11873 11874 11875 11876 11877 11878 11879]
[[    0]
 [    1]
 [    2]
 ...
 [11757]
 [11758]
 [11759]]
["{'cutting tools'}" "{'cutting tools'}" "{'cutting tools'}" ...
 "{'protection tools'}" "{'protection tools'}" "{'protection tools'}"]
TRAIN: [7140 2674 1813 ... 8404 2418 1304] TEST: [10544  4385  2555 ...  

(         filename  width  height             class  xmin  xmax  ymin  ymax
 7140   007141.jpg    960     540       clamp tools   389   424   120   260
 2674   002675.jpg    960     540    fastener tools   491   579   160   390
 1813   001814.jpg    960     540     cutting tools   337   395   203   333
 3636   003637.jpg    960     540    fastener tools   372   448   208   293
 3283   003284.jpg    960     540    fastener tools   259   312    88   259
 ...           ...    ...     ...               ...   ...   ...   ...   ...
 10898  010899.jpg    960     540  protection tools   373   410   281   300
 5659   005660.jpg    960     540   measuring tools   525   615   142   254
 8404   008405.jpg    960     540            marker   340   359   335   339
 2418   002419.jpg    960     540    fastener tools   390   401   296   309
 1304   001305.jpg    960     540     cutting tools   288   385   133   261
 
 [10584 rows x 8 columns],
          filename  width  height             class  xmin  

Map class labels function

In [ ]:
from object_detection.utils import label_map_util

__category_index = label_map_util.create_category_index_from_labelmap("{}/label_map.pbtxt".format(labels_path), use_display_name=True)

def class_text_to_int(label,category_index=__category_index):
  return list(filter(lambda x: x['name'] == label, list(category_index.values())))[0]['id']

Group classes between file splits

In [ ]:
def split(df, group):
  from collections import namedtuple
  Data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [Data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [ ]:
if(mount_dataset):
  import random
  grouped = split(dataframe, 'filename')

  i = random.randint(0,1000)
  for group in grouped[i::1000]:
    from PIL import Image
    filename = group.filename
    detections = get_detections(group)
    image_np = show_bb(filename,detections)
    display(Image.fromarray(image_np))
    print(group.object[['filename', 'class', 'xmin','xmax','ymin','ymax']])
else:
  print("Skipping dataset bounding box verification")

dataframe


Skipping dataset bounding box verification


,filename,width,height,class,xmin,xmax,ymin,ymax
0,000001.jpg,960,540,cutting tools,423,547,170,264
1,000002.jpg,960,540,cutting tools,446,554,178,266
2,000003.jpg,960,540,cutting tools,445,555,189,264
3,000004.jpg,960,540,cutting tools,449,561,201,276
4,000005.jpg,960,540,cutting tools,447,559,210,287
...,...,...,...,...,...,...,...,...
11875,011765.jpg,960,540,cutting tools,248,296,363,427
11876,011765.jpg,960,540,cutting tools,210,247,201,277
11877,011765.jpg,960,540,measuring tools,176,212,197,307
11878,011765.jpg,960,540,fastener tools,172,264,317,350


Create an example for TF Record function

In [ ]:
def create_tf_example(group, path):
  import io
  from PIL import Image
  from models.research.object_detection.utils import dataset_util

  with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    encoded_jpg = fid.read()
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = Image.open(encoded_jpg_io)
  width, height = image.size

  filename = group.filename.encode('utf8')
  image_format = b'jpg'
  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  classes_text = []
  classes = []

  for index, row in group.object.iterrows():
    xmin = row['xmin'] / width
    xmax = row['xmax'] / width
    ymin = row['ymin'] / height
    ymax = row['ymax'] / height
    assert(xmin <= 1 and xmax <= 1 and ymin <= 1 and ymax <= 1),"BBox out of limits for image\nwidth: {}\nheight: {}\nrow: {}".format( width, height, row)
    xmins.append(xmin)
    xmaxs.append(xmax)
    ymins.append(ymin)
    ymaxs.append(ymax)
    classes_text.append(row['class'].encode('utf8'))
    classes.append(class_text_to_int(row['class']))

  tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_example

Create a whole TF Record function

In [ ]:
def create_tf_record(output_path, image_dir, examples, overwrite=False):
  import os

  if (os.path.exists(output_path) and not overwrite):
    print('TFRecords already exist: {}'.format(output_path))
  else:
    !touch {output_path}
    !truncate -s 0 {output_path}

    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

In [ ]:
#@title Create TF Record for train and evaluation
overwrite = False #@param{type:'boolean'}
trainrec_fname = "train.record"
trainrec_path = "{}/{}".format(data_path, trainrec_fname)

valrec_fname = "val.record"
valrec_path = "{}/{}".format(data_path, valrec_fname)

create_tf_record(valrec_path, images_path, val_dataframe, overwrite)
create_tf_record(trainrec_path, images_path, train_dataframe, overwrite)

TFRecords already exist: /usr/local/lib/python3.6/dist-packages/tensorflow/models/data/val.record
TFRecords already exist: /usr/local/lib/python3.6/dist-packages/tensorflow/models/data/train.record


In [ ]:
!cat models/model/pipeline.config

# SSDLite with Mobilenet v3 small feature extractor.
# Trained on COCO14, initialized from scratch.
# TPU-compatible.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 90
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: tr

In [ ]:
# Change into the models directory
# Begin training
!cd {models_path} && \
python research/object_detection/model_main.py \
  --model_dir=model\out \
  --pipeline_config_path=model/pipeline.config \
  --sample_1_of_n_eval_examples=1 \
  --alsologtostderr
# !cd {models_path} && \
#   python {models_path}/research/object_detection/legacy/train.py \
#     --logtostderr \
#     --train_dir=model \
#     --pipeline_config_path=pipeline.config
